# Sample rooftops
Sample an additional 100 pins from Agra and Indore. 


**Contents:**  
1. Import packages, set paths, and import rooftop data
2. Sample 50 rooftops per district
3. Randomly sort the rooftops and assign each a number
4. Create a single map showing all of the points
5. Save in a new folder with date and time

## 1. Import packages, set paths, and import rooftop data

In [ ]:
from datetime import datetime
from pathlib import Path

import folium
import geopandas as gpd
import pandas as pd

In [ ]:
# set path to main data folder
FH_SAMPLING_FOLDER = Path("../") / "data" / "fortify_data"

# create a new folder in Samples with current date and time. save the output here
current_date = datetime.now().strftime("%Y-%m-%d")
SAMPLE_OUTPUT = FH_SAMPLING_FOLDER / "Samples" / current_date
SAMPLE_OUTPUT.mkdir(exist_ok=True)

In [ ]:
rooftops_5_dists = gpd.read_parquet(
    FH_SAMPLING_FOLDER / "Cleaned rooftop data" / "Rooftops in 5 pilot dists.parquet"
)
rooftops_indore = rooftops_5_dists.loc[rooftops_5_dists["d_name"] == "Indore"]

rooftops_agra_solan = gpd.read_parquet(
    FH_SAMPLING_FOLDER / "Cleaned rooftop data" / "Rooftops in Agra and Solan.parquet"
)
rooftops_agra = rooftops_agra_solan.loc[rooftops_agra_solan["d_name"] == "Agra"]
rooftops = pd.concat([rooftops_indore, rooftops_agra])

# confirm that rooftops is a gdf
assert isinstance(rooftops, gpd.GeoDataFrame)

# confirm that the dataset only has Indore and Agra
rooftops.d_name.value_counts()

## 2. Sample N rooftops per district

In [ ]:
sampled_rooftops = rooftops.groupby("d_name").sample(n=100, random_state=8675309)
sampled_rooftops.d_name.value_counts()

In [ ]:
# random assign the rooftops numbers from 1 to N within each group
sampled_rooftops = sampled_rooftops.sample(frac=1, random_state=42).reset_index(
    drop=True
)
sampled_rooftops["row_num"] = sampled_rooftops.groupby("d_name").cumcount() + 1

In [ ]:
sampled_rooftops[["d_name", "row_num", "geometry"]].sort_values(
    ["d_name", "row_num"]
).to_csv(SAMPLE_OUTPUT / "sampled_rooftops.csv", index=False)

## 3. Create map

In [ ]:
center_lat = sampled_rooftops.geometry.y.mean()
center_long = sampled_rooftops.geometry.x.mean()

m = folium.Map(
    location=[center_lat, center_long],
    zoom_start=6,  # Increased zoom level for a closer view
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community",
)

for _, row in sampled_rooftops.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        icon=folium.DivIcon(
            html=f'<div style="font-size: 10pt; color: #ADD8E6">{row.row_num}</div>'
        ),
    ).add_to(m)
m.save(SAMPLE_OUTPUT / "rooftop_map.html")